In [1]:
import cv2
import numpy as np
import os
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
import numpy as np


C:\Users\hassn\anaconda3\envs\newenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
id_image = 'ID_image.jpg'


def extract_id_photo(image_path):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    for (x, y, w, h) in faces:
        extracted_photo = image[y:y + h, x:x + w]
        return extracted_photo


In [6]:
# test the extraction function
extracted_face = extract_id_photo('ID_image2.jpg')
cv2.imshow('Extracted Photo', extracted_face)
cv2.waitKey(0)
cv2.destroyAllWindows() 

# saving extracted face as a temporary image file
cv2.imwrite('temp_face.jpg', extracted_face)


True

In [7]:
# feature extraction and matching
mtcnn = MTCNN(image_size=160, margin=0, keep_all=True, device='cuda' if torch.cuda.is_available() else 'cpu')
resnet = InceptionResnetV1(pretrained='vggface2').eval()  

def get_embedding(image):
    # preprocessing the image to match the input requirements of the MTCNN and InceptionResnetV1 models
    face = mtcnn(image)
    embedding = resnet(face) 
    return embedding

def cosine_similarity(embedding1, embedding2):
    if embedding1.shape == (1, 512):
        embedding1 = embedding1.reshape(-1)
    if embedding2.shape == (1, 512):
        embedding2 = embedding2.reshape(-1)

    sim = np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))
    return sim

In [8]:
# testing the matching function 
temp_face_image = cv2.imread('temp_face.jpg')
embedding_id = get_embedding(temp_face_image)
other_image = cv2.imread('image2.jpg', 0)
other_image = cv2.cvtColor(other_image, cv2.COLOR_GRAY2BGR)
embedding_other = get_embedding(other_image)
similarity = cosine_similarity(embedding_id.detach().numpy(), embedding_other.detach().numpy())
print(f"Cosine Similarity: {similarity}")

Cosine Similarity: 0.7862959504127502
